In [69]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_excel('AKL單元冷凍系統操作條件(20100101_20141231).xlsx',index_col=0,skiprows=2)
for i in df.columns:
  df[i] = pd.to_numeric(df[i],errors='coerce')
df = df.iloc[1:,:]
df = df.dropna(axis=1,how='all').dropna(axis=0)
df.head()

,入口溫度,出口溫度,入口壓力,出口壓力,出口溫度.1,出口流量,硫酸濃度
2014/12/25,8.1,66.2,0.1,4.8,28.5,231.4,91.98
2014/12/26,7.9,66.2,0.1,4.8,28.4,231.1,91.66
2014/12/27,8.2,66.3,0.1,4.8,28.4,232.2,91.95
2014/12/28,8.0,66.1,0.1,4.8,28.4,237.8,91.89
2014/12/29,7.6,65.7,0.2,4.8,28.4,245.5,92.12


In [70]:
df.max(),df.min()

(入口溫度        8.2
 出口溫度       66.3
 入口壓力        0.2
 出口壓力        4.8
 出口溫度.1     28.5
 出口流量      249.4
 硫酸濃度       92.4
 dtype: float64,
 入口溫度        7.20
 出口溫度       65.30
 入口壓力        0.10
 出口壓力        4.80
 出口溫度.1     28.30
 出口流量      231.10
 硫酸濃度       91.66
 dtype: float64)

In [71]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler,MinMaxScaler
真理 = 42
model = SVR(C=真理,gamma=真理)
ss = StandardScaler()
#mm = MinMaxScaler()
x_col = ['入口溫度','出口溫度','入口壓力','出口壓力','出口溫度.1','出口流量']
y_col = '硫酸濃度'
df_ss = df.copy()
df_ss[:] = ss.fit_transform(df_ss[:])
print(df_ss.shape)
df_ss

(7, 7)


,入口溫度,出口溫度,入口壓力,出口壓力,出口溫度.1,出口流量,硫酸濃度
2014/12/25,0.758747,0.591608,-0.866025,0.0,1.870829e+00,-0.885339,-0.236269
2014/12/26,0.133897,0.591608,-0.866025,0.0,-6.646519e-14,-0.929606,-1.666652
2014/12/27,1.071172,0.887412,-0.866025,0.0,-6.646519e-14,-0.767294,-0.370367
2014/12/28,0.446322,0.295804,-0.866025,0.0,-6.646519e-14,0.059023,-0.638564
2014/12/29,-0.803379,-0.887412,1.154701,0.0,-6.646519e-14,1.195208,0.389524
2014/12/30,-2.053080,-2.070628,1.154701,0.0,-1.870829e+00,1.770679,0.881218
2014/12/31,0.446322,0.591608,1.154701,0.0,-6.646519e-14,-0.442670,1.641109


In [72]:
df_ss[y_col].max()

1.6411091602363723

In [73]:
x,y = df_ss[x_col],df_ss[y_col]
model.fit(x,y)
model.score(x,y)

0.9899675302029113

In [74]:
df_ss.loc[:,x_col[0]].min()

-2.0530804763162833

In [75]:
import optuna
import numpy as np
request = {} 
request['output'] = df_ss[y_col].max()
def objective(trial: optuna.Trial):
  Input_df = pd.DataFrame(
    index = [0],
    columns = x_col
    )
  for key in x_col:
    Input_df.loc[0,key] = trial.suggest_uniform(
      name = key,
      low = max(df_ss.loc[:,key].min(),0),
      high = df_ss.loc[:,key].max()
      )
  output_df = model.predict(Input_df)
  output = output_df[0]
  total_loss = np.sqrt((request['output']-output)**2) #L1 LOSS
  return 1/total_loss # 求最大值

study = optuna.create_study()
study.optimize(objective, n_trials=真理)

[I 2022-06-22 11:57:43,053] A new study created in memory with name: no-name-24c7775d-026a-4e0c-8797-58a5a6e6abc5
[I 2022-06-22 11:57:43,060] Trial 0 finished with value: 0.6146948536358788 and parameters: {'入口溫度': 0.6377955771168098, '出口溫度': 0.13362969456394047, '入口壓力': 1.1094495004618405, '出口壓力': 0.0, '出口溫度.1': 0.5550389807291667, '出口流量': 0.8809090964406419}. Best is trial 0 with value: 0.6146948536358788.
[I 2022-06-22 11:57:43,065] Trial 1 finished with value: 0.6146948536358788 and parameters: {'入口溫度': 0.34404611736248114, '出口溫度': 0.5982374659523557, '入口壓力': 1.1179434903607512, '出口壓力': 0.0, '出口溫度.1': 1.4531102933080007, '出口流量': 1.5805540829295608}. Best is trial 0 with value: 0.6146948536358788.
[I 2022-06-22 11:57:43,070] Trial 2 finished with value: 0.6146948536358788 and parameters: {'入口溫度': 0.823112047689154, '出口溫度': 0.8435454225268278, '入口壓力': 0.9954343721430313, '出口壓力': 0.0, '出口溫度.1': 0.4779779366758955, '出口流量': 0.41244297411102254}. Best is trial 0 with value: 0.61469485363

In [76]:
study.best_params

{'入口溫度': 0.6377955771168098,
 '出口溫度': 0.13362969456394047,
 '入口壓力': 1.1094495004618405,
 '出口壓力': 0.0,
 '出口溫度.1': 0.5550389807291667,
 '出口流量': 0.8809090964406419}

In [77]:
model.predict(pd.DataFrame(study.best_params,index=[0]))[0]

0.014285714285804971

In [78]:
df_answer = pd.DataFrame()
for i in study.best_params:
    df_answer[i] = [study.best_params[i]]
df_answer[y_col] = model.predict(pd.DataFrame(study.best_params,index=[0]))[0]
df_answer[:] = ss.inverse_transform(df_answer[:])
print('r2_score:',model.score(x,y))
print('answer:')
df_answer

r2_score: 0.9899675302029113
answer:


,入口溫度,出口溫度,入口壓力,出口壓力,出口溫度.1,出口流量,硫酸濃度
0,8.061286,66.045175,0.197761,4.8,28.429668,243.369976,92.036053
